# Multi Cloud Service Networking with Consul - Base

### THIS TUTORIAL IS UNDER CONSTRUCTION

THIS TUTORIAL IS UNDER CONSTRUCTION - MORE TO COME

This tutorial covers setting up your environment for the Multi-Cloud Demo. Here are the things it will cover.

* Use Packer to build images in AWS, Azure, and Google.
* Use Terraform to provision infrastructure in all three clouds.
  * Network, Storage, Kubernetes

## Prerequisites

* Azure Client
* AWS Client

### Set Main Environment Variables

In [2]:
export RED="\e[0;31m" YELLOW="\e[0;33m" BLDYELLOW="\e[1;33m" GREEN="\e[0;32m"
export CYAN="\e[0;36m" BLUE="\e[0;34m" WHITE="\e[0;37m" BLDWHITE="\e[1;37m"
export NC="\e[0m"
# Some commands may have sensitive information. Prevent commands starting with a space to be saved to shell history.
HISTCONTROL=ignoreboth # do not save lines that begin with space in history

printf "${GREEN}# Set MAIN_DIR for future reference.${NC}\n"
export MAIN_DIR=${PWD}

# Set MAIN_DIR for future reference.l


: 1

Be sure to replace `POC_NAME` with your own value.

In [3]:
export POC_NAME=pphan
export GIT_REPO_DIR=config/consul/field-workshops-consul
export WORK_DIR=$MAIN_DIR/$GIT_REPO_DIR
printf "$MAIN_DIR\n$GIT_REPO_DIR\n$WORK_DIR"

/Users/pephan/OneDrive/code/vault_transit_test/hc_demos-jupyter/Consul
config/consul/field-workshops-consul
/Users/pephan/OneDrive/code/vault_transit_test/hc_demos-jupyter/Consul/config/consul/field-workshops-consul

: 1

Be sure to replace `TF_ORG` and `TF_WS_NAME` with your own values.

In [ ]:
export TF_GIT_DIR="config/terraform/<repo>"
printf "${GREEN}# Set Terraform Variables.${NC}\n"
printf "\n$MAIN_DIR \n $TF_GIT_DIR \n"

Be sure to replace `VAULT_ADDR` with your own values.

In [ ]:
printf "${GREEN}# Set Vault Variables.${NC}\n"
export VAULT_PORT=8200
export VAULT_TOKEN=root
export VAULT_ADDR=http://localhost:${VAULT_PORT}
export VAULT_LICENSE=$(cat ../../license/vault.hclic)
# Set dir env vars. Create required directory.
WORK_DIR=config/vault
mkdir -p config/vault/{data,logs,config}

### Cloud Credentials

NOTE: You can grab credentials from Instruqt

In [ ]:
```
env | grep -i ^AWS_ | grep -v ACCOUNT && \
env | grep INSTRUQT_AZURE_SUBSCRIPTION_CONSUL_SUBSCRIPTION_ID && \
env | grep INSTRUQT_AZURE_SUBSCRIPTION_CONSUL_SPN_ID \
&& env | grep INSTRUQT_AZURE_SUBSCRIPTION_CONSUL_SPN_PASSWORD
```

#### AWS Credentials

`terraform` will use credentials set in your environment or through other means as described in the [Terraform documentation](https://www.terraform.io/docs/providers/aws/index.html#environment-variables). This guide will assume you are using the "**Environment Variables**" method.

Add your AWS credentials as two environment variables. Set your `AWS_ACCESS_KEY_ID` and `AWS_SECRET_ACCESS_KEY` replacing `AAAAAA` with your own values.

In [ ]:
 export AWS_ACCESS_KEY_ID=AAAAAA
 export AWS_SECRET_ACCESS_KEY=AAAAAA
#  export AWS_SESSION_TOKEN=AAAAAA

NOTE: If you use `doormat`, then use this method.

In [1]:
alias awscredsenv='doormat --smoke-test || doormat -r && eval $(doormat aws -a $account)'
awscredsenv

Doormat auth credentials are valid!


: 1

In [ ]:
#debugging
echo $AZURE_CREDS_FILE
echo $INSTRUQT_AZURE_SUBSCRIPTION_CONSUL_SUBSCRIPTION_ID
echo $INSTRUQT_AZURE_SUBSCRIPTION_CONSUL_SPN_ID
echo $INSTRUQT_AZURE_SUBSCRIPTION_CONSUL_SPN_PASSWORD

In [ ]:
export AZURE_CREDS_FILE=${HOME}/.Azure/creds2.txt
export INSTRUQT_AZURE_SUBSCRIPTION_CONSUL_SUBSCRIPTION_ID="REPLACEME"
export INSTRUQT_AZURE_SUBSCRIPTION_CONSUL_SPN_ID=$(jq -r .appId ${AZURE_CREDS_FILE})
export INSTRUQT_AZURE_SUBSCRIPTION_CONSUL_SPN_PASSWORD=$(jq -r .password ${AZURE_CREDS_FILE})

In [ ]:
export GOOGLE_CREDENTIALS=$(echo $INSTRUQT_GCP_PROJECT_CONSUL_SERVICE_ACCOUNT_KEY | base64 -d)

# echo 'export GOOGLE_CREDENTIALS=$(echo $INSTRUQT_GCP_PROJECT_CONSUL_SERVICE_ACCOUNT_KEY | base64 -d)' >> ${HOME}/.bashrc
# source ${HOME}/.bashrc

### Install Cloud Clients

In [ ]:
brew update
brew install awscli
brew install azure-cli

Login to Azure CLI

In [ ]:
az login
# az login \
#   --username "${INSTRUQT_AZURE_SUBSCRIPTION_CONSUL_USERNAME}" \
#   --password "${INSTRUQT_AZURE_SUBSCRIPTION_CONSUL_PASSWORD}"

### AWS Setup

In [ ]:
aws configure set default.region us-east-1
aws ec2 create-default-vpc

### Azure Resource Group for Packer

In [ ]:
az group create -l westus2 -n ${POC_NAME}-packer

### jwt validation

In [ ]:
# apt install npm -y
# npm install -g jwt-cli

### Clone Assets

In [ ]:
git clone https://github.com/hashicorp/field-workshops-consul.git $GIT_REPO_DIR

In [ ]:
cp -r $GIT_REPO_DIR/instruqt-tracks/multi-cloud-service-networking-with-consul/assets/terraform .
cp -r $GIT_REPO_DIR/instruqt-tracks/multi-cloud-service-networking-with-consul/assets/packer .
cp -r $GIT_REPO_DIR/instruqt-tracks/multi-cloud-service-networking-with-consul/assets/apps .
cp -r $GIT_REPO_DIR/instruqt-tracks/multi-cloud-service-networking-with-consul/assets/policies .
cp -r $GIT_REPO_DIR/instruqt-tracks/multi-cloud-service-networking-with-consul/assets/helm .

Open folder in VS Code.

In [ ]:
code .

## Packer

In [ ]:
echo "Provisoning Packer Builds..."
# cd $MAIN_DIR/packer
pushd packer
nohup packer build hashistack.json 2>&1 | tee nohup.out &
popd

This can take awhile (~7 minutes). Go grab some coffee. You can check the status with the code editor.

**NOTE**: Need to fix.
* Azure account variables.
* License files.
* more

## Terraform

#### Provisoning Infra

In [ ]:
# cd ${MAIN_DIR}/terraform/infra
terraform -chdir=terraform/infra init
pubkey=$(cat ~/.ssh/id_rsa.pub)
cat > $MAIN_DIR/terraform/infra/terraform.tfvars << EOF
gcp_project_id="$(echo $INSTRUQT_GCP_PROJECT_CONSUL_PROJECT_ID)"
ssh_public_key="${pubkey}"
EOF

In [ ]:
terraform -chdir=terraform/infra apply -auto-approve 2>&1 | tee terraform/infra/nohup.out

#### Provisoning IAM

In [4]:
#iam
echo "Provisoning IAM..."
cd $MAIN_DIR/terraform/iam
terraform -chdir=$MAIN_DIR/terraform/iam init
cat > $MAIN_DIR/terraform/iam/terraform.tfvars << EOF
gcp_project_id="$(echo $INSTRUQT_GCP_PROJECT_CONSUL_PROJECT_ID)"
EOF

Provisoning IAM...

Initializing the backend...

Initializing provider plugins...
- terraform.io/builtin/terraform is built in to Terraform
- Finding hashicorp/google versions matching "~> 3.45.0"...
- Finding hashicorp/aws versions matching "~> 3.0"...
- Finding hashicorp/azurerm versions matching "2.47.0"...
- Installing hashicorp/google v3.45.0...
- Installed hashicorp/google v3.45.0 (signed by HashiCorp)
- Installing hashicorp/aws v3.59.0...
- Installed hashicorp/aws v3.59.0 (signed by HashiCorp)
- Installing hashicorp/azurerm v2.47.0...
- Installed hashicorp/azurerm v2.47.0 (signed by HashiCorp)

Terraform has created a lock file .terraform.lock.hcl to record the provider
selections it made above. Include this file in your version control repository
so that Terraform can guarantee to make the same selections by default when
you run "terraform init" in the future.

╷
│ Warning: Version constraints inside provider configuration blocks are deprecated
│ 
│   on main.tf line 2, in prov

: 1

In [ ]:
terraform -chdir=$MAIN_DIR apply -auto-approve 2>&1 | tee $MAIN_DIR/nohup.out

#### Provisoning GKE K8s Consul Cluster

In [6]:
#gcp GKE secondary
echo "Provisoning GKE K8s Consul Cluster..."
# cd $MAIN_DIR/terraform/gcp-consul-secondary
terraform -chdir=$MAIN_DIR/terraform/gcp-consul-secondary init
cat > $MAIN_DIR/terraform/gcp-consul-secondary/terraform.tfvars <<EOF
gcp_project_id="$(echo $INSTRUQT_GCP_PROJECT_CONSUL_PROJECT_ID)"
EOF

Provisoning GKE K8s Consul Cluster...

Initializing the backend...

Initializing provider plugins...
- terraform.io/builtin/terraform is built in to Terraform
- Finding hashicorp/google versions matching "~> 3.45.0"...
- Finding latest version of hashicorp/google-beta...
- Installing hashicorp/google v3.45.0...
- Installed hashicorp/google v3.45.0 (signed by HashiCorp)
- Installing hashicorp/google-beta v3.84.0...
- Installed hashicorp/google-beta v3.84.0 (signed by HashiCorp)

Terraform has created a lock file .terraform.lock.hcl to record the provider
selections it made above. Include this file in your version control repository
so that Terraform can guarantee to make the same selections by default when
you run "terraform init" in the future.

╷
│ Warning: Version constraints inside provider configuration blocks are deprecated
│ 
│   on main.tf line 2, in provider "google":
│    2:   version = "~> 3.45.0"
│ 
│ Terraform 0.13 and earlier allowed provider version constraints inside the

: 1

In [ ]:
export GOOGLE_CREDENTIALS=$(echo $INSTRUQT_GCP_PROJECT_CONSUL_SERVICE_ACCOUNT_KEY | base64 -d)
nohup terraform -chdir=$MAIN_DIR/terraform/gcp-consul-secondary apply -auto-approve 2>&1 | tee $MAIN_DIR/terraform/gcp-consul-secondary/nohup.out &

#### Provisoning GKE K8s App Clusters

In [7]:
#gcp GKE app clusters
echo "Provisoning GKE K8s App Clusters..."
# cd $MAIN_DIR/terraform/k8s-scheduler-services
terraform -chdir=$MAIN_DIR/terraform/k8s-scheduler-services init
cat > $MAIN_DIR/terraform/k8s-scheduler-services/terraform.tfvars <<EOF
gcp_project_id="$(echo $INSTRUQT_GCP_PROJECT_CONSUL_PROJECT_ID)"
EOF

Provisoning GKE K8s App Clusters...

Initializing the backend...

Initializing provider plugins...
- terraform.io/builtin/terraform is built in to Terraform
- Finding latest version of hashicorp/google-beta...
- Finding hashicorp/google versions matching "~> 3.45.0"...
- Finding latest version of hashicorp/kubernetes...
- Installing hashicorp/google-beta v3.84.0...
- Installed hashicorp/google-beta v3.84.0 (signed by HashiCorp)
- Installing hashicorp/google v3.45.0...
- Installed hashicorp/google v3.45.0 (signed by HashiCorp)
- Installing hashicorp/kubernetes v2.5.0...
- Installed hashicorp/kubernetes v2.5.0 (signed by HashiCorp)

Terraform has created a lock file .terraform.lock.hcl to record the provider
selections it made above. Include this file in your version control repository
so that Terraform can guarantee to make the same selections by default when
you run "terraform init" in the future.

╷
│ Warning: Version constraints inside provider configuration blocks are deprecated
│ 


: 1

In [ ]:
export GOOGLE_CREDENTIALS=$(echo $INSTRUQT_GCP_PROJECT_CONSUL_SERVICE_ACCOUNT_KEY | base64 -d)
nohup terraform -chdir=$MAIN_DIR/terraform/k8s-scheduler-services apply -auto-approve 2>&1 | tee $MAIN_DIR/terraform/k8s-scheduler-services/nohup.out &

#### Provisoning AWS ElastiCache

In [ ]:
#aws elasticache
echo "Provisoning AWS ElastiCache..."
cd $MAIN_DIR/terraform/cache-services
terraform init
AWS_CONSUL_IP=$(terraform output -state /root/terraform/aws-consul-primary/terraform.tfstate aws_consul_public_ip)
cat <<EOF > $MAIN_DIR/terraform/cache-services/terraform.tfvars
consul_http_addr="http://127.0.0.1:8500"
EOF
nohup terraform apply -target=aws_elasticache_cluster.redis -auto-approve 2>&1 | tee nohup.out &

#### Provisoning Azure PostgreSQL

In [ ]:
#azure postgreSQL
echo "Provisoning Azure PostgreSQL..."
cd $MAIN_DIR/terraform/database-services
terraform init
AZURE_CONSUL_IP=$(terraform output -state /root/terraform/azure-consul-secondary/terraform.tfstate azure_consul_public_ip)
cat <<EOF > $MAIN_DIR/terraform/database-services/terraform.tfvars
consul_http_addr="http://127.0.0.1:8500"
EOF
nohup terraform apply -target=azurerm_postgresql_server.postgres -auto-approve 2>&1 | tee nohup.out &

## Clean Up

### Remove Cloud Assets

### Azure

Remove Azure Resource Group for Packer

In [ ]:
az group delete -y --no-wait -n ${POC_NAME}-packer

## Terraform Destroy

In [ ]:
terraform -chdir=terraform/infra plan -destroy 

In [ ]:
terraform -chdir=terraform/infra destroy -auto-approve

### Remove Assets

In [ ]:
pushd $MAIN_DIR
rm -rf $GIT_REPO_DIR apps helm packer policies terraform nohup.out
popd

# Misc

### Packer

In [ ]:
The terraform code will provision cloud infrastructure in AWS, GPC, and Azure.

Your lab environment will leverage pre-built packer images.
You can inspect the image build in the code editor, and validate the images are available in AWS & Azure.

Validate the images are available in AWS & Azure

In [ ]:
az image list -g packer | jq
# aws ec2 describe-images --owners self | jq

In [ ]:
docker run --rm -d \
    --name consul \
    -p 8500:8500 \
    -e 'VAULT_DEV_ROOT_TOKEN_ID=root' \
    -e "VAULT_ADDR=http://127.0.0.1:8200" \
    hashicorp/consul